<a href="https://colab.research.google.com/github/jafprofesor/Aprendizaje-profundo/blob/main/Aprendizaje_Profundo_desde_Cero_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Configuración Inicial**
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

%matplotlib inline
print("✅ Librerías cargadas!")

### **Sección 1: Perceptrón y Compuerta Lógica**  

**Conceptos**:  
- **Perceptrón**: Unidad básica de una red neuronal. Toma entradas $x_i$, aplica pesos $w_i$, suma un sesgo $b$, y pasa el resultado por una función de activación.  
- **Función Escalón**: Activación binaria (0 o 1) para decisiones simples.  
- **Aprendizaje**: Ajuste de pesos mediante `error = etiqueta_real - predicción`.

**Implementación:**

In [ ]:

import numpy as np

class Perceptron:
    def __init__(self, n_inputs, lr=0.1):
        self.weights = np.zeros(n_inputs)  # Inicializar pesos en 0
        self.bias = 0                      # Inicializar sesgo en 0
        self.lr = lr                       # Tasa de aprendizaje

    def predict(self, inputs):
        """Predicción: inputs · weights + bias"""
        z = np.dot(inputs, self.weights) + self.bias
        return 1 if z >= 0 else 0  # Función escalón

    def train(self, X, y, epochs):
        """Entrenamiento: Ajustar pesos con el error"""
        for _ in range(epochs):
            for inputs, label in zip(X, y):
                y_pred = self.predict(inputs)
                error = label - y_pred  # Cálculo del error
                # Actualizar pesos y sesgo
                self.weights += self.lr * error * inputs
                self.bias += self.lr * error

# Datos: Compuerta AND
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([0, 0, 0, 1])

# Entrenar perceptrón
p = Perceptron(n_inputs=2, lr=0.1)
p.train(X, y, epochs=10)

# Probar
print(p.predict([1, 1]))  # Debe devolver 1
```

---


### **Sección 2: Propagación hacia Adelante y Funciones de Activación**  # Nueva sección

**Conceptos**:  
- **Propagación hacia Adelante**: Cálculo de salidas capa por capa usando $a^{(l)} = f(W^{(l)} \cdot a^{(l-1)} + b^{(l)})$.  
- **Funciones de Activación**:  
  - **Sigmoid**: $f(z) = \frac{1}{1 + e^{-z}}$ (para probabilidades).  
  - **ReLU**: $f(z) = \max(0, z)$ (evita desvanecimiento de gradientes).  

**Implementación**:


In [ ]:
def sigmoid(x):
    """Función sigmoid: 1 / (1 + e^(-x))"""
    return 1 / (1 + np.exp(-x))

def relu(x):
    """Función ReLU: max(0, x)"""
    return np.maximum(0, x)

def forward_pass(X, weights, biases):
    """Cálculo de salidas de la red"""
    # Capa oculta: X · W_hidden + b_hidden
    hidden_input = np.dot(X, weights['hidden']) + biases['hidden']
    hidden_output = relu(hidden_input)  # Activación ReLU

    # Capa salida: hidden_output · W_output + b_output
    final_input = np.dot(hidden_output, weights['output']) + biases['output']
    return sigmoid(final_input)  # Activación sigmoid

# Ejemplo: Red con 3 neuronas ocultas
np.random.seed(42)
weights = {
    'hidden': np.random.randn(2, 3),  # 2 entradas, 3 ocultas
    'output': np.random.randn(3, 1)   # 3 ocultas, 1 salida
}
biases = {'hidden': np.zeros(3), 'output': 0}

# Ejecutar propagación
X_sample = np.array([[0.5, 0.8]])
output = forward_pass(X_sample, weights, biases)
print("Salida de la red:", output)
```

---


### **Sección 3: Función de Pérdida y Retropropagación**  

**Conceptos**:  
- **Error Cuadrático Medio (MSE)**: $\text{MSE} = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$.  
- **Retropropagación**:  
  1. Calcular gradiente de la pérdida respecto a salidas.  
  2. Propagarlo hacia atrás usando regla de la cadena.  
  3. Actualizar pesos con $\Delta W = -\alpha \frac{\partial \text{Pérdida}}{\partial W}$.  

**Implementación**:


In [ ]:
def mse_loss(y_true, y_pred):
    """Error cuadrático medio"""
    return np.mean((y_true - y_pred) ** 2)

def backprop(X, y_true, weights, biases, lr=0.1):
    # Paso 1: Forward pass (guardar valores intermedios)
    hidden_input = np.dot(X, weights['hidden']) + biases['hidden']
    hidden_output = relu(hidden_input)
    output = sigmoid(np.dot(hidden_output, weights['output']) + biases['output'])

    # Paso 2: Calcular gradientes
    d_output = (output - y_true) * output * (1 - output)  # Derivada de sigmoid
    d_hidden = np.dot(d_output, weights['output'].T) * (hidden_output > 0)  # Derivada de ReLU

    # Paso 3: Actualizar parámetros
    weights['output'] -= lr * np.dot(hidden_output.T, d_output)
    biases['output'] -= lr * np.sum(d_output)
    weights['hidden'] -= lr * np.dot(X.T, d_hidden)
    biases['hidden'] -= lr * np.sum(d_hidden, axis=0)

# Ejemplo de uso
X_sample = np.array([[0, 0]])
y_true = np.array([[0]])
backprop(X_sample, y_true, weights, biases, lr=0.1)
```

---


### **Sección 4: Descenso de Gradiente Estocástico (SGD)**  

**Conceptos**:  
- **SGD**: Actualiza pesos con mini-lotes de datos en lugar de todo el conjunto.  
- **Ventajas**: Más rápido y evita mínimos locales.  

**Implementación**:


In [ ]:
def sgd(X, y, weights, biases, lr=0.01, batch_size=32, epochs=100):
    n_samples = len(X)
    for epoch in range(epochs):
        # Mezclar datos en cada época
        indices = np.random.permutation(n_samples)
        for i in range(0, n_samples, batch_size):
            batch_idx = indices[i:i+batch_size]
            X_batch, y_batch = X[batch_idx], y[batch_idx]
            backprop(X_batch, y_batch, weights, biases, lr)
```

---


### Visualización del Descenso de Gradiente  

Vamos a crear una representación gráfica que muestre cómo el algoritmo de descenso de gradiente encuentra el mínimo de una función de pérdida. Usaremos una función cuadrática simple para ilustrar el proceso.

#### **Conceptos Clave**:  
- **Función de Pérdida**: Representa el error del modelo (ej. $J(w) = w^2$).  
- **Gradiente**: Derivada de la función ($\nabla J(w) = 2w$).  
- **Actualización de Pesos**: $w_{\text{nuevo}} = w_{\text{antiguo}} - \alpha \nabla J(w)$.  

#### **Implementación con Visualización**:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Configuración inicial
plt.style.use('seaborn-whitegrid')
%matplotlib inline

# 1. Definir función de pérdida y su gradiente
def loss_function(w):
    return w**2  # J(w) = w²

def gradient(w):
    return 2*w  # ∇J(w) = 2w

# 2. Parámetros del descenso de gradiente
w_start = 5.0    # Peso inicial
learning_rate = 0.2
epochs = 15

# 3. Almacenar historial de pesos y pérdidas
history_w = [w_start]
history_loss = [loss_function(w_start)]

# 4. Ejecutar descenso de gradiente
w = w_start
for _ in range(epochs):
    grad = gradient(w)  # Calcular gradiente
    w = w - learning_rate * grad  # Actualizar peso
    history_w.append(w)
    history_loss.append(loss_function(w))

# 5. Crear gráfico animado
fig, ax = plt.subplots(figsize=(10, 6))
w_range = np.linspace(-6, 6, 100)
ax.plot(w_range, loss_function(w_range), label='J(w) = w²')
ax.set_xlabel('Peso (w)', fontsize=12)
ax.set_ylabel('Pérdida', fontsize=12)
ax.set_title('Descenso de Gradiente', fontsize=15)
point, = ax.plot([], [], 'ro', markersize=10)
path, = ax.plot([], [], 'r--', alpha=0.5)

def init():
    point.set_data([], [])
    path.set_data([], [])
    return point, path

def animate(i):
    # Mostrar punto actual y trayectoria
    current_w = history_w[i]
    current_loss = history_loss[i]
    point.set_data([current_w], [current_loss])

    # Dibujar trayectoria completa
    path.set_data(history_w[:i+1], history_loss[:i+1])

    # Añadir flecha de gradiente
    if i > 0:
        ax.annotate('',
                    xy=(history_w[i], history_loss[i]),
                    xytext=(history_w[i-1], history_loss[i-1]),
                    arrowprops=dict(arrowstyle='->', color='gray', alpha=0.7))

    return point, path

# 6. Generar animación
anim = FuncAnimation(fig, animate, frames=len(history_w),
                     init_func=init, blit=True, interval=800)
plt.close()

# Mostrar en notebook
HTML(anim.to_html5_video())
```


#### **Interpretación del Gráfico**:  
1. **Curva Roja**: Función de pérdida $J(w) = w^2$ (con mínimo en $w=0$).  
2. **Punto Rojo**: Valor actual del peso ($w$) y su pérdida asociada.  
3. **Línea Discontinua**: Trayectoria del descenso de gradiente.  
4. **Flechas**: Dirección del movimiento en cada paso.  

#### **Efecto de la Tasa de Aprendizaje**:  


In [ ]:
# Comparar diferentes tasas de aprendizaje
learning_rates = [0.01, 0.2, 0.9]
colors = ['blue', 'green', 'purple']

plt.figure(figsize=(10, 6))
w_range = np.linspace(-6, 6, 100)
plt.plot(w_range, loss_function(w_range), 'k-', label='J(w) = w²')

for lr, color in zip(learning_rates, colors):
    w = 5.0
    history_w = [w]
    for _ in range(15):
        w = w - lr * gradient(w)
        history_w.append(w)
    plt.plot(history_w, loss_function(np.array(history_w)),
             'o--', color=color, alpha=0.7, label=f'LR = {lr}')

plt.xlabel('Peso (w)')
plt.ylabel('Pérdida')
plt.title('Efecto de la Tasa de Aprendizaje')
plt.legend()
plt.grid(True)
```


![Efecto de la Tasa de Aprendizaje](https://i.imgur.com/8ZQDq7L.png)

#### **Conclusiones Visuales**:  
- **LR = 0.01**: Convergencia muy lenta (pasos pequeños).  
- **LR = 0.2**: Convergencia óptima (llega al mínimo en pocos pasos).  
- **LR = 0.9**: Oscila y diverge (pasos demasiado grandes).  


### **Sección 5: Regularización**  

**Conceptos**:  
- **Overfitting**: Modelo memoriza datos de entrenamiento pero no generaliza.  
- **Regularización L2**: Penaliza pesos grandes: $\text{Pérdida} + \lambda \sum w_i^2$.  
- **Dropout**: Apaga neuronas aleatoriamente durante entrenamiento.  

**Implementación**:


In [ ]:
# L2 Regularization
def l2_regularization(weights, lambda_val=0.01):
    penalty = 0
    for key in weights:
        penalty += np.sum(weights[key]**2)
    return lambda_val * penalty

# Dropout en forward pass
def forward_dropout(X, weights, dropout_rate=0.5):
    mask = (np.random.rand(*X.shape) > dropout_rate) / (1 - dropout_rate)
    return np.dot(X * mask, weights['hidden'])
```

---


### **Sección 6: Redes Recurrentes (RNN) y BPTT**  

**Conceptos**:  
- **RNN**: Neuronas con conexiones recurrentes para datos secuenciales.  
- **BPTT**: Backpropagation Through Time: Retropropagación desenrollada en pasos temporales.  

**Implementación**:


In [ ]:
class SimpleRNN:
    def __init__(self, input_size, hidden_size):
        self.Wx = np.random.randn(hidden_size, input_size) * 0.01  # Pesos para entrada
        self.Wh = np.random.randn(hidden_size, hidden_size) * 0.01 # Pesos recurrentes
        self.b = np.zeros(hidden_size)                             # Sesgo

    def forward(self, X):
        """Propagación para una secuencia X"""
        h = [np.zeros(self.Wh.shape[0])]  # Estado oculto inicial
        for t in range(len(X)):
            h_t = np.tanh(np.dot(self.Wx, X[t]) + np.dot(self.Wh, h[-1]) + self.b)
            h.append(h_t)
        return h

# BPTT: Requiere guardar todos los estados para calcular gradientes temporales
```

---
